In [1]:
!git clone https://github.com/GNNatan/SemNov_AML_DAAI_23-24.git

Cloning into 'SemNov_AML_DAAI_23-24'...
remote: Enumerating objects: 1050, done.
remote: Counting objects: 100% (515/515), done.
remote: Compressing objects: 100% (302/302), done.
remote: Total 1050 (delta 241), reused 404 (delta 202), pack-reused 535
Receiving objects: 100% (1050/1050), 3.26 GiB | 22.73 MiB/s, done.
Resolving deltas: 100% (313/313), done.
Updating files: 100% (734/734), done.


In [1]:
%cd SemNov_AML_DAAI_23-24/

/content/SemNov_AML_DAAI_23-24


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.5/881.5 kB 49.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-

In [2]:
!sh download_data.sh

============Downloading ModelNet40 + OOD Splits in 
--2024-06-11 16:34:06--  https://www.dropbox.com/s/c2x3h59nxprjs21/modelnet40_normal_resampled.tar?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/c2x3h59nxprjs21/modelnet40_normal_resampled.tar [following]
--2024-06-11 16:34:07--  https://www.dropbox.com/s/dl/c2x3h59nxprjs21/modelnet40_normal_resampled.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5601b6b581f18ca5fe6cf96dec.dl.dropboxusercontent.com/cd/0/get/CUpp2bo3sf0BpRwh1c5uWw12p9gEwm2lAxre6dXl7q7tMpeyEoafM-jD8LdS_a8wkcBTucGC_IoZCsS-YGeq8vL0vvk88Fxb65U1jVG0kZlUN0UKdySIw1SgDkwqbNJjPnLDf93qy0rFmFuh6PzKe1Ow/file?dl=1# [following]
--2024-06-11 16:34:08--  https://uc5601b6b581f18ca5fe6cf96dec.dl.dropboxusercontent.com/

In [3]:
import torch
import os
import h5py
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from random import sample
from mpl_toolkits.mplot3d import proj3d
from notebooks.utils import *
from utils import ood_utils

Cannot import torchlars


In [4]:
SR1 = {
    4: 0,  # chair
    8: 1,  # shelf
    7: 2,  # door
    12: 3,  # sink
    13: 4  # sofa
}

SR2 = {
    10: 0,  # bed
    14: 1,  # toilet
    5: 2,  # desk
    6: 3,  # display
    9: 2  # table
}

SR3 = {
    0: 404,  # bag
    1: 404,  # bin
    2: 404,  # box
    3: 404,  # cabinet
    11: 404  # pillow
}

test_file = h5py.File('3D_OS_release_data/ScanObjectNN/h5_files/main_split/test_objectdataset.h5', 'r')
train_file = h5py.File('3D_OS_release_data/ScanObjectNN/h5_files/main_split/training_objectdataset.h5', 'r')

test_labels  = test_file['label'][:]
train_labels = train_file['label'][:]

test_clouds = test_file['data'][:]
train_clouds = train_file['data'][:]

all_labels = np.hstack((train_labels, test_labels))
all_clouds = np.vstack((train_clouds, test_clouds))

SR1_indices = [index for index, value in enumerate(all_labels) if value in SR1.keys()]
SR2_indices = [index for index, value in enumerate(all_labels) if value in SR2.keys()]
SR3_indices = [index for index, value in enumerate(all_labels) if value in SR3.keys()]

SR1_clouds = all_clouds[SR1_indices]
SR2_clouds = all_clouds[SR2_indices]
SR3_clouds = all_clouds[SR3_indices]

SR1_labels = [SR1[all_labels[id]] for id in SR1_indices]
SR2_labels = [SR2[all_labels[id]] for id in SR2_indices]
SR3_labels = [404 for _ in SR3_indices]

In [5]:
## OPENSHAPE - MAXIMUM SOFTMAX PROBABILITY
sr1_img_scores = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR1/img_scores.pt')
sr2_img_scores = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR2/img_scores.pt')
sr1_txt_scores = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR1/text_scores.pt')
sr2_txt_scores = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR2/text_scores.pt')


openshape_sr1_scores = sr1_img_scores + sr1_txt_scores
openshape_sr2_scores = sr2_img_scores + sr2_txt_scores

sr1_src_logits = torch.tensor(np.array([openshape_sr1_scores[i] for i in SR1_indices])).squeeze(1)
sr1_tar1_logits = torch.tensor(np.array([openshape_sr1_scores[i] for i in SR2_indices])).squeeze(1)
sr1_tar2_logits = torch.tensor(np.array([openshape_sr1_scores[i] for i in SR3_indices])).squeeze(1)

sr1_src_MSP_scores, sr1_src_MSP_pred = sr1_src_logits.max(1)
sr1_tar1_MSP_scores, sr1_tar1_MSP_pred = sr1_tar1_logits.max(1)
sr1_tar2_MSP_scores, sr1_tar2_MSP_pred = sr1_tar2_logits.max(1)


sr2_src_logits = torch.tensor(np.array([openshape_sr2_scores[i] for i in SR2_indices])).squeeze(1)
sr2_tar1_logits = torch.tensor(np.array([openshape_sr2_scores[i] for i in SR1_indices])).squeeze(1)
sr2_tar2_logits = torch.tensor(np.array([openshape_sr2_scores[i] for i in SR3_indices])).squeeze(1)

# merge desk and table prediction into same class for SR2
sr2_src_table = sr2_src_logits[:, 2] + sr2_src_logits[:, 4]
sr2_tar1_table = sr2_tar1_logits[:, 2] + sr2_tar1_logits[:, 4]
sr2_tar2_table = sr2_tar2_logits[:, 2] + sr2_tar2_logits[:, 4]

sr2_src_logits = torch.cat((sr2_src_logits[:, :2], sr2_src_table.unsqueeze(1), sr2_src_logits[:, 3].unsqueeze(1)), dim=1)
sr2_tar1_logits = torch.cat((sr2_tar1_logits[:, :2], sr2_tar1_table.unsqueeze(1), sr2_tar1_logits[:, 3].unsqueeze(1)), dim=1)
sr2_tar2_logits = torch.cat((sr2_tar2_logits[:, :2], sr2_tar2_table.unsqueeze(1), sr2_tar2_logits[:, 3].unsqueeze(1)), dim=1)


sr2_src_MSP_scores, sr2_src_MSP_pred = sr2_src_logits.max(1)
sr2_tar1_MSP_scores, sr2_tar1_MSP_pred = sr2_tar1_logits.max(1)
sr2_tar2_MSP_scores, sr2_tar2_MSP_pred = sr2_tar2_logits.max(1)

In [6]:
ood_utils.eval_ood_sncore(
    scores_list = [sr1_src_MSP_scores, sr1_tar1_MSP_scores, sr1_tar2_MSP_scores],
    preds_list = [sr1_src_MSP_pred, None, None],
    labels_list = [SR1_labels, None, None],
    src_label = 1
  )

AUROC - Src label: 1, Tar label: 0
Src Test - Clf Acc: 0.8589641434262948, Clf Bal Acc: 0.8378403945607398
SRC->TAR1:      AUROC: 0.8682, FPR95: 0.7449, AUPR_IN: 0.9241, AUPR_OUT: 0.7439
SRC->TAR2:      AUROC: 0.8458, FPR95: 0.6021, AUPR_IN: 0.8848, AUPR_OUT: 0.7880
SRC->TAR1+TAR2: AUROC: 0.8566, FPR95: 0.6709, AUPR_IN: 0.8363, AUPR_OUT: 0.8661
to spreadsheet: 0.8682452929399154,0.7449238578680203,0.9240610777104156,0.7438754143772683,0.8458200256823943,0.602125147579693,0.8847589164368889,0.7879571993315008,0.8566280443973342,0.6709480122324158,0.8362922890684257,0.8660990804528985


(0.8589641434262948,
 0.8378403945607398,
 {'fpr_at_95_tpr': 0.7449238578680203,
  'detection_error': 0.4766561553057779,
  'auroc': 0.8682452929399154,
  'aupr_in': 0.9240610777104156,
  'aupr_out': 0.7438754143772683},
 {'fpr_at_95_tpr': 0.602125147579693,
  'detection_error': 0.3794057492453425,
  'auroc': 0.8458200256823943,
  'aupr_in': 0.8847589164368889,
  'aupr_out': 0.7879571993315008},
 {'fpr_at_95_tpr': 0.6709480122324158,
  'detection_error': 0.31931234038692585,
  'auroc': 0.8566280443973342,
  'aupr_in': 0.8362922890684257,
  'aupr_out': 0.8660990804528985})

In [7]:
ood_utils.eval_ood_sncore(
    scores_list = [sr2_src_MSP_scores, sr2_tar1_MSP_scores, sr2_tar2_MSP_scores],
    preds_list = [sr2_src_MSP_pred, None, None],
    labels_list = [SR2_labels, None, None],
    src_label = 1
  )

AUROC - Src label: 1, Tar label: 0
Src Test - Clf Acc: 0.5203045685279187, Clf Bal Acc: 0.30559854397948993
SRC->TAR1:      AUROC: 0.5875, FPR95: 0.9641, AUPR_IN: 0.4817, AUPR_OUT: 0.6510
SRC->TAR2:      AUROC: 0.5577, FPR95: 0.9374, AUPR_IN: 0.5191, AUPR_OUT: 0.5642
SRC->TAR1+TAR2: AUROC: 0.5755, FPR95: 0.9534, AUPR_IN: 0.3321, AUPR_OUT: 0.7594
to spreadsheet: 0.587483568265011,0.9641434262948207,0.4816855573357774,0.6510431857346664,0.5576684506079983,0.9374262101534829,0.519082694128402,0.5641997510658865,0.5754695793708674,0.9533777354900095,0.3320659469930958,0.7593592786485661


(0.5203045685279187,
 0.30559854397948993,
 {'fpr_at_95_tpr': 0.9641434262948207,
  'detection_error': 0.38509262021907475,
  'auroc': 0.587483568265011,
  'aupr_in': 0.4816855573357774,
  'aupr_out': 0.6510431857346664},
 {'fpr_at_95_tpr': 0.9374262101534829,
  'detection_error': 0.47589707899102784,
  'auroc': 0.5576684506079983,
  'aupr_in': 0.519082694128402,
  'aupr_out': 0.5641997510658865},
 {'fpr_at_95_tpr': 0.9533777354900095,
  'detection_error': 0.2717563434395978,
  'auroc': 0.5754695793708674,
  'aupr_in': 0.3320659469930958,
  'aupr_out': 0.7593592786485661})

In [16]:
# OPENSHAPE - DISTANCE BASED EVALUATION

sr1_train_labels = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR1/train_labels.pt')
sr1_src_labels = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR1/src_labels.pt')
sr1_src_dist = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR1/src_dist.pt')
sr1_src_ids = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR1/src_ids.pt')
sr1_tar1_dist = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR1/tar1_dist.pt')
sr1_tar1_ids = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR1/tar1_ids.pt')
sr1_tar2_dist = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR1/tar2_dist.pt')
sr1_tar2_ids = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR1/tar2_dist.pt')


sr2_train_labels = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR2/train_labels.pt')
sr2_src_labels = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR2/src_labels.pt')
sr2_src_dist = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR2/src_dist.pt')
sr2_src_ids = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR2/src_ids.pt')
sr2_tar1_dist = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR2/tar1_dist.pt')
sr2_tar1_ids = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR2/tar1_ids.pt')
sr2_tar2_dist = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR2/tar2_dist.pt')
sr2_tar2_ids = load_from_file('/content/SemNov_AML_DAAI_23-24/outputs/tensors/OpenShape_SR2/tar2_dist.pt')

In [12]:
sr1_src_scores = 1 / sr1_src_dist
sr1_src_pred = np.asarray([sr1_train_labels[i] for i in sr1_src_ids])  # pred is label of nearest training sample
sr1_tar1_scores = 1 / sr1_tar1_dist
sr1_tar2_scores = 1 / sr1_tar2_dist

In [13]:
ood_utils.eval_ood_sncore(
        scores_list=[sr1_src_scores, sr1_tar1_scores, sr1_tar2_scores],
        preds_list=[sr1_src_pred, None, None],
        labels_list=[sr1_src_labels, None, None],
        src_label=1  # confidence should be higher for ID samples
    )

AUROC - Src label: 1, Tar label: 0
Src Test - Clf Acc: 0.9139442231075697, Clf Bal Acc: 0.917365080048558
SRC->TAR1:      AUROC: 0.7842, FPR95: 0.8185, AUPR_IN: 0.8686, AUPR_OUT: 0.6414
SRC->TAR2:      AUROC: 0.8087, FPR95: 0.8135, AUPR_IN: 0.8794, AUPR_OUT: 0.6769
SRC->TAR1+TAR2: AUROC: 0.7969, FPR95: 0.8159, AUPR_IN: 0.7947, AUPR_OUT: 0.7930
to spreadsheet: 0.7841517180010921,0.8185279187817259,0.8685865952839535,0.6414371478091789,0.8087348363335325,0.8134592680047226,0.8793713072348379,0.6769219669952151,0.7968868257855428,0.8159021406727829,0.7947343079708069,0.7930141626375626


(0.9139442231075697,
 0.917365080048558,
 {'fpr_at_95_tpr': 0.8185279187817259,
  'detection_error': 0.5212559201389987,
  'auroc': 0.7841517180010921,
  'aupr_in': 0.8685865952839535,
  'aupr_out': 0.6414371478091789},
 {'fpr_at_95_tpr': 0.8134592680047226,
  'detection_error': 0.5049407256186057,
  'auroc': 0.8087348363335325,
  'aupr_in': 0.8793713072348379,
  'aupr_out': 0.6769219669952151},
 {'fpr_at_95_tpr': 0.8159021406727829,
  'detection_error': 0.3813579644365513,
  'auroc': 0.7968868257855428,
  'aupr_in': 0.7947343079708069,
  'aupr_out': 0.7930141626375626})

In [17]:
sr2_src_scores = 1 / sr2_src_dist
sr2_src_pred = np.asarray([sr2_train_labels[i] for i in sr2_src_ids])  # pred is label of nearest training sample
sr2_tar1_scores = 1 / sr2_tar1_dist
sr2_tar2_scores = 1 / sr2_tar2_dist

In [18]:
ood_utils.eval_ood_sncore(
        scores_list=[sr2_src_scores, sr2_tar1_scores, sr2_tar2_scores],
        preds_list=[sr2_src_pred, None, None],
        labels_list=[sr2_src_labels, None, None],
        src_label=1  # confidence should be higher for ID samples
    )

AUROC - Src label: 1, Tar label: 0
Src Test - Clf Acc: 0.5647208121827412, Clf Bal Acc: 0.730759124314575
SRC->TAR1:      AUROC: 0.6684, FPR95: 0.8255, AUPR_IN: 0.5621, AUPR_OUT: 0.7620
SRC->TAR2:      AUROC: 0.5275, FPR95: 0.9634, AUPR_IN: 0.5588, AUPR_OUT: 0.5185
SRC->TAR1+TAR2: AUROC: 0.6117, FPR95: 0.8811, AUPR_IN: 0.4003, AUPR_OUT: 0.8029
to spreadsheet: 0.6684439905353207,0.8254980079681274,0.5621351264676989,0.7620142858987206,0.5275037606601981,0.9634002361275088,0.5588437021340469,0.5185239463689529,0.6116521852526239,0.8810656517602283,0.40033548514621436,0.8028990916596447


(0.5647208121827412,
 0.730759124314575,
 {'fpr_at_95_tpr': 0.8254980079681274,
  'detection_error': 0.3488034138234562,
  'auroc': 0.6684439905353207,
  'aupr_in': 0.5621351264676989,
  'aupr_out': 0.7620142858987206},
 {'fpr_at_95_tpr': 0.9634002361275088,
  'detection_error': 0.47955409274211325,
  'auroc': 0.5275037606601981,
  'aupr_in': 0.5588437021340469,
  'aupr_out': 0.5185239463689529},
 {'fpr_at_95_tpr': 0.8810656517602283,
  'detection_error': 0.26171343791472856,
  'auroc': 0.6116521852526239,
  'aupr_in': 0.40033548514621436,
  'aupr_out': 0.8028990916596447})